## Day 14

### Part 1 

As your ferry approaches the sea port, the captain asks for your help again. The computer system that runs this port isn't compatible with the docking program on the ferry, so the docking parameters aren't being correctly initialized in the docking program's memory.

After a brief inspection, you discover that the sea port's computer system uses a strange bitmask system in its initialization program. Although you don't have the correct decoder chip handy, you can emulate it in software!

The initialization program (your puzzle input) can either update the bitmask or write a value to memory. Values and memory addresses are both 36-bit unsigned integers. For example, ignoring bitmasks for a moment, a line like mem[8] = 11 would write the value 11 to memory address 8.

The bitmask is always given as a string of 36 bits, written with the most significant bit (representing 2^35) on the left and the least significant bit (2^0, that is, the 1s bit) on the right. The current bitmask is applied to values immediately before they are written to memory: a 0 or 1 overwrites the corresponding bit in the value, while an X leaves the bit in the value unchanged.

For example, consider the following program:

- mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
- mem[8] = 11
- mem[7] = 101
- mem[8] = 0

This program starts by specifying a bitmask (mask = ....). The mask it specifies will overwrite two bits in every written value: the 2s bit is overwritten with 0, and the 64s bit is overwritten with 1.

The program then attempts to write the value 11 to memory address 8. By expanding everything out to individual bits, the mask is applied as follows:

- value:  000000000000000000000000000000001011  (decimal 11)
- mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
- result: 000000000000000000000000000001001001  (decimal 73)

So, because of the mask, the value 73 is written to memory address 8 instead. Then, the program tries to write 101 to address 7:

- value:  000000000000000000000000000001100101  (decimal 101)
- mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
- result: 000000000000000000000000000001100101  (decimal 101)

This time, the mask has no effect, as the bits it overwrote were already the values the mask tried to set. Finally, the program tries to write 0 to address 8:

- value:  000000000000000000000000000000000000  (decimal 0)
- mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
- result: 000000000000000000000000000001000000  (decimal 64)

64 is written to address 8 instead, overwriting the value that was there previously.

To initialize your ferry's docking program, you need the sum of all values left in memory after the initialization program completes. (The entire 36-bit address space begins initialized to the value 0 at every address.) In the above example, only two values in memory are not zero - 101 (at address 7) and 64 (at address 8) - producing a sum of 165.

Execute the initialization program. What is the sum of all values left in memory after it completes?

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('input_data/Day14.txt', sep='=', header=None)
df.columns=['action','value']
df['action']=df['action'].str.strip('\n')
df['action']=df['action'].str.strip(' ')
df['value']=df['value'].str.strip('\n')
df['value']=df['value'].str.strip(' ')
df.head()

,action,value
0,mask,X111000X0101100001000000100011X0000X
1,mem[4812],133322396
2,mem[39136],1924962
3,mem[35697],29912136
4,mem[41065],2558851


In [3]:
def process_address(val, mask):
    '''takes a value instruction and a mask and returns the result'''
   
    # convert our new value to binary, strip the leading 0b and pad it with zeros on the left
    val = ( 36*'0' + (bin(int(val))[2:]) )[-36:]
    result = val

    for i in range(36):       

        if mask[i] != 'X':
            result = result[:i] + mask[i] + result[i+1:]
        
    return result

In [4]:
# Create a new dataframe to hold memory values
memory = pd.DataFrame(columns=['value'])

# Looop through our dataframe
for idx, row in df.iterrows():
    
    if row['action']=='mask':
      
        # we have a new mask
        mask = row['value'] 
#        print('new mask: ', mask)
    
    else: # we have a new value for an address
        
        # process next instruction
        address = int(row['action'][row['action'].find('[')+1:].strip(']'))
        #print('address:', address)
        
        if not (address in memory.index):
            # initialize this address
            memory.loc[address] = 36*'0'
            
        memory.loc[address] = process_address(row['value'], mask)

In [5]:
# Let's sum all the values in our memory
memory.apply(lambda x: int(x['value'],2), axis=1).sum()

8570568288597

### Part 2


For some reason, the sea port's computer system still can't communicate with your ferry's docking program. It must be using version 2 of the decoder chip!

A version 2 decoder chip doesn't modify the values being written at all. Instead, it acts as a memory address decoder. Immediately before a value is written to memory, each bit in the bitmask modifies the corresponding bit of the destination memory address in the following way:

- If the bitmask bit is 0, the corresponding memory address bit is unchanged.
- If the bitmask bit is 1, the corresponding memory address bit is overwritten with 1.
- If the bitmask bit is X, the corresponding memory address bit is floating.

A floating bit is not connected to anything and instead fluctuates unpredictably. In practice, this means the floating bits will take on all possible values, potentially causing many memory addresses to be written all at once!

For example, consider the following program:

- mask = 000000000000000000000000000000X1001X
- mem[42] = 100
- mask = 00000000000000000000000000000000X0XX
- mem[26] = 1

When this program goes to write to memory address 42, it first applies the bitmask:

- address: 000000000000000000000000000000101010  (decimal 42)
- mask:    000000000000000000000000000000X1001X
- result:  000000000000000000000000000000X1101X

After applying the mask, four bits are overwritten, three of which are different, and two of which are floating. Floating bits take on every possible combination of values; with two floating bits, four actual memory addresses are written:

- 000000000000000000000000000000011010  (decimal 26)
- 000000000000000000000000000000011011  (decimal 27)
- 000000000000000000000000000000111010  (decimal 58)
- 000000000000000000000000000000111011  (decimal 59)

Next, the program is about to write to memory address 26 with a different bitmask:

- address: 000000000000000000000000000000011010  (decimal 26)
- mask:    00000000000000000000000000000000X0XX
- result:  00000000000000000000000000000001X0XX

This results in an address with three floating bits, causing writes to eight memory addresses:

- 000000000000000000000000000000010000  (decimal 16)
- 000000000000000000000000000000010001  (decimal 17)
- 000000000000000000000000000000010010  (decimal 18)
- 000000000000000000000000000000010011  (decimal 19)
- 000000000000000000000000000000011000  (decimal 24)
- 000000000000000000000000000000011001  (decimal 25)
- 000000000000000000000000000000011010  (decimal 26)
- 000000000000000000000000000000011011  (decimal 27)

The entire 36-bit address space still begins initialized to the value 0 at every address, and you still need the sum of all values left in memory at the end of the program. In this example, the sum is 208.

Execute the initialization program using an emulator for a version 2 decoder chip. What is the sum of all values left in memory after it completes?

In [6]:
# Let's look at all the masks
df[df['action']=='mask']['value'].str.count('X').max()

9

In [7]:
df[df['action']!='mask']['action'].count()

467

In [8]:
# So at most we have 512 addresses to write for each instruction, and we have 467 instructions, so that's
467*512

239104

In [9]:
# so we shouldn't have performance issues (I don't think!) and we can just
# use simple logic

In [10]:
# Let's add a column to just store addresses so we don't have process these every time
def get_addr(address):
    '''takes mem string and returns address'''
    if address=='mask':
        return address
    else:
        return int(address[address.find('[')+1:].strip(']'))

In [11]:
#df.apply(lambda x: get_addr(x['action']),axis = 1)
df['address']=df.apply(lambda x: get_addr(x['action']),axis = 1)

In [12]:
df

,action,value,address
0,mask,X111000X0101100001000000100011X0000X,mask
1,mem[4812],133322396,4812
2,mem[39136],1924962,39136
3,mem[35697],29912136,35697
4,mem[41065],2558851,41065
...,...,...,...
562,mem[48342],177911,48342
563,mask,00011X11011X010X00000011X10010X10111,mask
564,mem[6531],1493325,6531
565,mem[35058],21547,35058


In [13]:
def process_address(val, mask):
    '''takes a value instruction and a mask and returns the result'''
   
    # convert our new value to binary, strip the leading 0b and pad it with zeros on the left
    val = ( 36*'0' + (bin(int(val))[2:]) )[-36:]
    result = val

    for i in range(36):       

        if mask[i] != 'X':
            result = result[:i] + mask[i] + result[i+1:]
        
    return result

In [14]:
def get_addresses(addr, mask):
    '''takes an address and a mask and returns a list of all possible addresses'''
    
    # convert our new address to binary, strip the leading 0b and pad it with zeros on the left
    addr = ( 36*'0' + (bin(addr)[2:]) )[-36:]
    
    # apply the mask to the address
    for i in range(36):       
        # if we have a X or 1 replace that bit with X or 1, otherwise leave it
        if mask[i] in ['1','X']:
            addr = addr[:i] + mask[i] + addr[i+1:] 

    # initialize a list to hold all the variations of the address
    working_addr = addr
    addresses = [addr]
    i = working_addr.find('X')

    # loop through our working address as long as we have another X, replacing each X with Y as we deal with it
    while i >= 0:
    
        curr_num_addr = len(addresses)
        for j in range(curr_num_addr):
            st = addresses.pop(0)
            addresses.append(st[:i] + '1' + st[i+1:])
            addresses.append(st[:i] + '0' + st[i+1:])
    
        # replace 'X' with 'Y'
        working_addr = working_addr[:i] + 'Y' + working_addr[i+1:]
        i = working_addr.find('X')
       
    return addresses

In [15]:
# Test this for our two test cases
get_addresses(42,'000000000000000000000000000000X1001X')

['000000000000000000000000000000111011',
 '000000000000000000000000000000111010',
 '000000000000000000000000000000011011',
 '000000000000000000000000000000011010']

In [16]:
get_addresses(26,'00000000000000000000000000000000X0XX')

['000000000000000000000000000000011011',
 '000000000000000000000000000000011010',
 '000000000000000000000000000000011001',
 '000000000000000000000000000000011000',
 '000000000000000000000000000000010011',
 '000000000000000000000000000000010010',
 '000000000000000000000000000000010001',
 '000000000000000000000000000000010000']

In [17]:
# Create a new dataframe to hold memory values
memory = pd.DataFrame(columns=['value'])

# Looop through our dataframe
for idx, row in df.iterrows():
    
    if row['action']=='mask':
      
        # we have a new mask
        mask = row['value'] 
    
    else: # we have a new value for an address
        
        # Now, the value that is written to memory can just be the decimal value
        val = int(row['value'])
        
        # get the list of addresses affected
        addresses = get_addresses(int(row['address']), mask)
        
        # Now we write val to each of these memory locations
        for address in addresses:
            dec_addr = int(address,2)
            memory.loc[dec_addr] = val
            
# Let's sum all the values in our memory
memory['value'].sum()

3289441921203